In [109]:
import pandas as pd
import pickle
import xgboost as xgb

pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from xgboost import XGBRanker


## 读取数据

In [110]:
ratings_df=pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/ml-1m/ratings.csv')
movies_df=pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/ml-1m/movies.csv')
users_df=pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/ml-1m/users.csv')

In [111]:
# 需要的用户特征和电影特征列
user_features_list, movie_features_list=pickle.load(open('/Users/bytedance/Desktop/MovieLens-Recommendation-System/rank/config/feature_keep.p', 'rb'))

ratings_df=pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/ml-1m/ratings.csv')
movies_df=pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/ml-1m/movies.csv')

import json
def read_json_file(filepath):
    """
    读取指定路径的 JSON 文件。

    Args:
        filepath (str): JSON 文件的完整路径。

    Returns:
        dict or list or None: 如果成功读取，则返回 JSON 文件中的数据（通常是字典或列表）。
                                如果文件不存在或读取失败，则返回 None。
    """
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"错误: 文件 '{filepath}' 未找到。")
        return None
    except json.JSONDecodeError:
        print(f"错误: 文件 '{filepath}' 不是有效的 JSON 格式。")
        return None
    except Exception as e:
        print(f"读取文件 '{filepath}' 时发生未知错误: {e}")
        return None
    
movie_embedding_config=read_json_file('/Users/bytedance/Desktop/MovieLens-Recommendation-System/rank/config/movie_embedding_config.json')
user_embedding_config=read_json_file('/Users/bytedance/Desktop/MovieLens-Recommendation-System/rank/config/user_embedding_config.json')
with open('/Users/bytedance/Desktop/MovieLens-Recommendation-System/rank/features/movie_features.pkl', 'rb') as f:
    movie_features = pickle.load(f)
with open('/Users/bytedance/Desktop/MovieLens-Recommendation-System/rank/features/user_features.pkl', 'rb') as f:
    user_features=pickle.load(f)


In [112]:
data = pd.merge(ratings_df, user_features, on='user_id', how='left')  # 合并用户特征
data = pd.merge(data, movie_features, on='movie_id', how='left')   # 合并电影特征
data = data.sort_values(by=['user_id', 'timestamp'])
data['label'] = data.apply(lambda row: 1 if row['rating'] > row['mean_rating'] else 0, axis=1) # 生成正负样本
data['rank'] = data.groupby('user_id')['rating'].rank(ascending=False, method='first')

In [113]:
data.head()

,user_id,movie_id,rating,timestamp,mean_rating,rating_std,rating_count,rating_min,rating_max,rating_strictness,rating_variability,Action_favorite_degree,Adventure_favorite_degree,Animation_favorite_degree,Children's_favorite_degree,Comedy_favorite_degree,Crime_favorite_degree,Documentary_favorite_degree,Drama_favorite_degree,Fantasy_favorite_degree,Film-Noir_favorite_degree,Horror_favorite_degree,Musical_favorite_degree,Mystery_favorite_degree,Romance_favorite_degree,Sci-Fi_favorite_degree,Thriller_favorite_degree,War_favorite_degree,Western_favorite_degree,num_liked_genres,activity_level_encoded,favorite_genre_encoded,gender_encoded,occupation_encoded,age_encoded,title,genres,movie_mean_rating,movie_rating_std,movie_rating_count,popularity,year,is_old_movie,genre_purity,title_length,label,rank
31,1,3186,4,978300019,4.188679,-1.615782,-0.584221,3.199606,0.061461,-0.607115,0.162573,0.043103,0.043103,0.155172,0.172414,0.12069,0.017241,0.0,0.181034,0.025862,0.0,0.0,0.12069,0.0,0.051724,0.025862,0.025862,0.017241,0.0,-1.097206,1,7,0,10,0,"[3849, 497, 2521, 2521, 2521, 2521, 2521, 2521...","[7, 7, 7, 7, 7, 7]",0.355312,0.001947,0.419564,3,5,1,-0.826985,-0.017184,0,19.0
22,1,1270,5,978300055,4.188679,-1.615782,-0.584221,3.199606,0.061461,-0.607115,0.162573,0.043103,0.043103,0.155172,0.172414,0.12069,0.017241,0.0,0.181034,0.025862,0.0,0.0,0.12069,0.0,0.051724,0.025862,0.025862,0.017241,0.0,-1.097206,1,7,0,10,0,"[4033, 2743, 3492, 1624, 2521, 2521, 2521, 252...","[4, 14, 4, 4, 4, 4]",1.116812,-0.426831,6.023789,3,1,1,0.405841,0.083903,1,1.0
27,1,1721,4,978300055,4.188679,-1.615782,-0.584221,3.199606,0.061461,-0.607115,0.162573,0.043103,0.043103,0.155172,0.172414,0.12069,0.017241,0.0,0.181034,0.025862,0.0,0.0,0.12069,0.0,0.051724,0.025862,0.025862,0.017241,0.0,-1.097206,1,7,0,10,0,"[2152, 2521, 2521, 2521, 2521, 2521, 2521, 252...","[7, 13, 7, 7, 7, 7]",0.512086,0.695404,3.323240,3,4,1,0.405841,-1.028060,0,20.0
37,1,1022,5,978300055,4.188679,-1.615782,-0.584221,3.199606,0.061461,-0.607115,0.162573,0.043103,0.043103,0.155172,0.172414,0.12069,0.017241,0.0,0.181034,0.025862,0.0,0.0,0.12069,0.0,0.051724,0.025862,0.025862,0.017241,0.0,-1.097206,1,7,0,10,0,"[3544, 2521, 2521, 2521, 2521, 2521, 2521, 252...","[2, 3, 11, 2, 2, 2]",0.768006,-0.035739,0.799776,3,0,1,1.638668,-0.724797,1,2.0
24,1,2340,3,978300103,4.188679,-1.615782,-0.584221,3.199606,0.061461,-0.607115,0.162573,0.043103,0.043103,0.155172,0.172414,0.12069,0.017241,0.0,0.181034,0.025862,0.0,0.0,0.12069,0.0,0.051724,0.025862,0.025862,0.017241,0.0,-1.097206,1,7,0,10,0,"[4499, 4501, 2552, 2521, 2521, 2521, 2521, 252...","[13, 13, 13, 13, 13, 13]",0.020830,0.314818,0.192999,2,4,1,-0.826985,-0.320447,0,46.0


In [114]:
data['popularity']=data['popularity'].astype(float)

## XGB模型

In [115]:
features = ['user_id', 'movie_id'] + list(user_features.columns[1:]) + list(movie_features.columns[1:])

features_used=[f for f in features if f not in ['title', 'genres']]
X = data[features_used]
y = data['rank']  

# 获取所有唯一的用户
unique_users = data['user_id'].unique()
print("总用户数为:", len(unique_users))

# 随机选择一些用户作为训练集，剩下的用户作为验证集
train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)
print("训练集用户数为:", len(train_users))
print("验证集用户数为:", len(test_users))
X_train=X[X['user_id'].isin(train_users)]
X_test=X[X['user_id'].isin(test_users)]
y_train=y[X['user_id'].isin(train_users)]
y_test=y[X['user_id'].isin(test_users)]
print("训练集样本量为:", len(X_train))
print("测试集样本量为:", len(X_test))


group_train = X_train.groupby('user_id').size().tolist()
group_test = X_test.groupby('user_id').size().tolist()
group=data.groupby('user_id').size().tolist()

总用户数为: 6040
训练集用户数为: 4832
验证集用户数为: 1208
训练集样本量为: 808218
测试集样本量为: 191991


In [116]:
# 检查训练集和测试集的用户是否有重叠
print("训练集和测试集用户重叠的数量:", len(set(train_users) & set(test_users)))

训练集和测试集用户重叠的数量: 0


In [117]:
len(group_train)

4832

In [118]:
model = XGBRanker(
    objective='rank:pairwise',
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    eval_metric='ndcg'
)

In [119]:
small_X_train = X_train.head(100)
small_y_train = y_train.head(100)
small_group_train = group_train[:100]
model.fit(
    small_X_train,
    small_y_train,
    group=group_train,  # 训练集的group
    eval_set=[(X_test, y_test)],  # 测试集
    eval_group=[group_test],  # 测试集的group
    verbose=10
)

XGBoostError: [21:58:34] /Users/runner/work/xgboost/xgboost/src/data/data.cc:762: Check failed: group_ptr_.back() == num_row_ (808218 vs. 100) : Invalid query group structure. The number of rows obtained from group doesn't equal to the actual number of rows given by data.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000040bfdc428 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000040c149890 xgboost::MetaInfo::Validate(xgboost::DeviceOrd) const + 572
  [bt] (2) 3   libxgboost.dylib                    0x000000040c1dacdc xgboost::LearnerConfiguration::InitBaseScore(xgboost::DMatrix const*) + 156
  [bt] (3) 4   libxgboost.dylib                    0x000000040c1c77a8 xgboost::LearnerImpl::UpdateOneIter(int, std::__1::shared_ptr<xgboost::DMatrix>) + 140
  [bt] (4) 5   libxgboost.dylib                    0x000000040bffeb34 XGBoosterUpdateOneIter + 144
  [bt] (5) 6   libffi.dylib                        0x0000000194d

## AutoInt模型

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

class AutoInt(tf.keras.Model):
    def __init__(self, user_embedding_config, movie_embedding_config, embedding_dim, num_heads, output_dim):
        super(AutoInt, self).__init__()
        
        # 电影特征的 Embedding 层
        self.movie_embeddings = {}
        for feature_name, config in movie_embedding_config.items():
            self.movie_embeddings[feature_name] = layers.Embedding(config['vocab_size'], config['embedding_dim'], input_length=config['seq_len'])
        
        # 用户特征的 Embedding 层
        self.user_embeddings = {}
        for feature_name, config in user_embedding_config.items():
            self.user_embeddings[feature_name] = layers.Embedding(config['vocab_size'], config['embedding_dim'], input_length=config['seq_len'])
        
        # 自注意力层
        self.attn_layer = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        
        # 合并用户和电影特征后通过全连接层进行评分预测
        self.fc = models.Sequential([
            layers.Dense(128, activation='relu'),
            layers.Dense(output_dim)  # 输出1维，用于评分预测
        ])
    
    def call(self, user_input, movie_input):
        # 获取电影特征的嵌入表示
        movie_embs = []
        for feature_name, feature_values in movie_input.items():
            movie_embs.append(self.movie_embeddings[feature_name](feature_values))
        
        # 合并所有电影特征的嵌入
        movie_emb = tf.concat(movie_embs, axis=-1)  # (batch_size, embedding_dim * feature_count)

        # 获取用户特征的嵌入表示
        user_embs = []
        for feature_name, feature_values in user_input.items():
            user_embs.append(self.user_embeddings[feature_name](feature_values))
        
        # 合并所有用户特征的嵌入
        user_emb = tf.concat(user_embs, axis=-1)  # (batch_size, embedding_dim * feature_count)

        # 将电影和用户特征拼接在一起
        combined_emb = tf.concat([movie_emb, user_emb], axis=-1)  # (batch_size, movie_embedding_dim + user_embedding_dim)

        # 自注意力层
        attn_output = self.attn_layer(combined_emb, combined_emb)

        # 通过全连接层进行评分预测
        output = self.fc(attn_output)
        return output


/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
auto_int_model = AutoInt(user_embedding_config, movie_embedding_config, embedding_dim=64, num_heads=4, output_dim=1)
auto_int_model.summary()

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "auto_int_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_13 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_14 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_15 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_16 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_17 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_18 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_19 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_20 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_21 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_22 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_23 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ multi_head_attention_1          │ ?                      │   0 (unbuilt) │
│ (MultiHeadAttention)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)